In [23]:
import os
import re
import json
import sys
import pandas as pd
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

In [2]:
dirs = ["biorxiv_medrxiv", "comm_use_subset", "noncomm_use_subset"]
articles = []

In [3]:
path = f"kaggle_data/{dirs[0]}/{dirs[0]}/pdf_json"
for file_ in tqdm(os.listdir(path)):
    with open(f"{path}/{file_}") as f:
        data = json.load(f)

        paper_id = data['paper_id']
        title = data['metadata']['title']

        full_abstract = ""
        for part in data['abstract']:
            full_abstract += part['text'] + "\n"

        full_text = ""
        for text_part in data["body_text"]:
            full_text += text_part["text"] + "\n\n"
        
        articles.append((paper_id, title, full_abstract, full_text))

100%|██████████| 1625/1625 [00:01<00:00, 1588.84it/s]


In [4]:
df = pd.DataFrame(articles, columns=['Paper ID', 'Title', 'Abstract', 'Text'])

In [5]:
df.head()

,Paper ID,Title,Abstract,Text
0,7fdd33661d188e3ea250e99f2752d705b5467ef9,Glycoinformatics approach for identifying targ...,COVID-19 outbreak is still threatening the pub...,The coronavirus spike protein (S) plays a key ...
1,ffbd7555a337706238c211197b221795e4e35146,Title: Estimation of COVID-2019 burden and pot...,,CC-BY-NC-ND 4.0 International license It is ma...
2,9a0c89a9b8ff3969d97b7cc4245489ff69f3a615,Development and external validation of a progn...,"Since the outbreak of COVID-19, there has been...",Background: COVID-19 pandemic has developed ra...
3,08911cdc65e71e6398ca79b46806e6c8b2b730ae,Epigenetic dysregulation of ACE2 and interfero...,medRxiv preprint immunology 2015; 156(1): 19-2...,Infection caused by SARS-CoV-2 can result in s...
4,bdaa40d95b82093f60a1c5ac8b798d67cef3a52b,A SARS-CoV-2 Vaccination Strategy Focused on P...,Here we propose a vaccination strategy for SAR...,The current SARS-CoV-2 pandemic has precipitat...


## Find all articles where 'incubator' word was used 

In [35]:
df_incubator = df[df['Text'].str.contains('incubator', flags=re.I)].copy()

def how_many_mentioned(text: str, keyword="incubator"):
    counter = 0
    for word in text.split():
        if word == keyword:
            counter += 1
    return counter

df_incubator["Mentioned"] = df_incubator["Text"].progress_apply(lambda text: text.lower().count("incubator"))

## Count words in abstract and title

In [38]:
df["Text Word Count"] = df["Text"].apply(lambda x: len(x.strip().split()))
df["Abstract Word Count"] = df["Abstract"].apply(lambda x: len(x.strip().split()))
df["Text Unique Word Count"] = df["Text"].apply(lambda x: len(set(x.strip().split())))

df.head()

,Paper ID,Title,Abstract,Text,Text Word Count,Abstract Word Count,Text Unique Word Count
0,7fdd33661d188e3ea250e99f2752d705b5467ef9,Glycoinformatics approach for identifying targ...,COVID-19 outbreak is still threatening the pub...,The coronavirus spike protein (S) plays a key ...,1876,175,660
1,ffbd7555a337706238c211197b221795e4e35146,Title: Estimation of COVID-2019 burden and pot...,,CC-BY-NC-ND 4.0 International license It is ma...,982,0,375
2,9a0c89a9b8ff3969d97b7cc4245489ff69f3a615,Development and external validation of a progn...,"Since the outbreak of COVID-19, there has been...",Background: COVID-19 pandemic has developed ra...,3711,237,1166
3,08911cdc65e71e6398ca79b46806e6c8b2b730ae,Epigenetic dysregulation of ACE2 and interfero...,medRxiv preprint immunology 2015; 156(1): 19-2...,Infection caused by SARS-CoV-2 can result in s...,1660,50,533
4,bdaa40d95b82093f60a1c5ac8b798d67cef3a52b,A SARS-CoV-2 Vaccination Strategy Focused on P...,Here we propose a vaccination strategy for SAR...,The current SARS-CoV-2 pandemic has precipitat...,4175,151,1120
